# Segmenting and Clustering Neighborhoods in Toronto
Applied Data Sciente Capstone - Week3 - Notebook2

## 1. Import Dependencies

In [ ]:
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # install
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # install
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 

## 2. Data Collection and Preprocessing

Scrap Wikipedia for Toronto's Neighborhood

In [ ]:
#Scrap Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))[0]
#Data Cleaning
df.columns=["PostalCode", "Borough", "Neighborhood"]
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)
df.reset_index(inplace=True,drop=True)
print(df.shape)

Import Geospatial data

In [ ]:
#import Geospatial Data
dfll = pd.read_csv('https://cocl.us/Geospatial_data')
dfll.columns=["PostalCode", "Latitude", "Longitude"]
dfll.set_index('PostalCode',inplace=True)
#Merge the data
dfToronto = pd.merge(df, dfll, on='PostalCode', how='inner')
print(dfToronto.shape)

## 3. Data Visualization on Toronto's Map

In [ ]:
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


## 4. Neighborhoods Exploration


#### Define Foursquare Credentials , Versions and Parameters

In [ ]:
CLIENT_ID = 'BG4TACIEB2U1YTBDC3PFEBCRJL32IADQKBPY21YW2TJSXT1R'  
CLIENT_SECRET = 'WBJOCDDP12DUL3ZCGZUEGXLVIOLZ3325GUBDQ4VXIIGSZRDF' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API


In [ ]:
#### Let's create a function to get nearby venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


#### Then use the function to create dataframe toronto_venues

In [ ]:
toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )
print(toronto_venues.shape)

Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))